In [1]:
from data_generators import get_iterator, get_dataset
from classifiers import theme_classifier

In [2]:
import torch
import torch.nn as nn

In [3]:
from torchtext.vocab import GloVe 
GLOVE_EMBEDDING = GloVe(name="6B", dim=300)

In [4]:
train_dataset, val_dataset, test_dataset, review_text_FIELD, theme_FIELD = get_dataset(vectors = 
                                                                                       GLOVE_EMBEDDING)

In [5]:
batch_size = 64
train_iter = get_iterator(train_dataset, batch_size, train=False, shuffle=False, repeat=False)
val_iter = get_iterator(val_dataset, batch_size, train=False, shuffle=False, repeat=False)
test_iter = get_iterator(test_dataset, batch_size, train=False, shuffle=False, repeat=False)

In [6]:
val_list = list(val_iter)

In [7]:
batch = val_list[50]
x = batch.review_text.transpose(1, 0).int()[:10]
y = batch.theme.int()

for idx in range(x.shape[0]):
    #print(x.shape, y.shape)
    print("{} | {}".format(' '.join([train_dataset.fields['review_text'].vocab.itos[_] for _ in x[idx]]),
         train_dataset.fields['theme'].vocab.itos[y[idx]]))

<sos> predictable and boring <eos> | other
<sos> very interesting . <eos> | other
<sos> not scary . <eos> | other
<sos> awful <unk> disturbing <eos> | other
<sos> same story . <eos> | plot
<sos> see this . <eos> | other
<sos> good movie . <eos> | other
<sos> worth the watch <eos> | other
<sos> what a ride <eos> | other
<sos> ¥ â ? <eos> | other


In [8]:
#review_text_FIELD.vocab.vectors.shape, len(review_text_FIELD.vocab.itos)

In [9]:
theme_FIELD.vocab.itos

['<unk>', 'other', 'plot', 'acting', 'effect', 'production']

In [10]:
vocab_size = review_text_FIELD.vocab.vectors.shape[0]
label_size = len(theme_FIELD.vocab) - 1
emb_dim = review_text_FIELD.vocab.vectors.shape[1]
vectors = train_dataset.fields["review_text"].vocab.vectors
hidden_dim = 1024
layers = 2
dropout = .5

vocab_size, label_size, emb_dim, vectors.shape

(20004, 5, 300, torch.Size([20004, 300]))

In [11]:
class BaseModel(nn.Module):

    def __init__(self, 
                 ninp = vocab_size, 
                 linp = label_size, 
                 emb_dim = emb_dim, 
                 emb_lab = 20,
                 nhid = hidden_dim, 
                 nout = vocab_size, 
                 nlayers = layers, 
                 dropout = dropout, 
                 vectors = vectors,
                 pretrained = False):
        super().__init__()
        
        self.ninp = ninp
        self.linp = linp
        self.emb_dim = emb_dim
        self.emb_lab = emb_lab
        self.nhid = nhid
        self.nout = nout
        self.nlayers = nlayers
        self.drop = nn.Dropout(dropout)

        self.word_embedding = nn.Embedding(ninp, emb_dim)
        self.label_embedding = nn.Embedding(linp, emb_lab)
        
        self.rnn = nn.LSTM(emb_dim + emb_lab, nhid, nlayers, dropout=dropout)
        self.rnn.flatten_parameters()
        self.decoder = nn.Linear(nhid, nout)
        #self.softmax = nn.Softmax(dim=-1)

        if pretrained:
            self.word_embedding.weight.data.copy_(vectors)
            self.word_embedding.from_pretrained(GLOVE_EMBEDDING.vectors)
            #init_embedding(self.embeddings, GLOVE_EMBEDDING)
            
        self.init_weights()    
            
    def init_weights(self):
        initrange = .1
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, reviews, labels, hidden):
        R = self.word_embedding(reviews)
        L = self.label_embedding(labels)
        L = torch.cat([L.unsqueeze(0)]*R.shape[0])
        X = torch.cat([R, L], -1)
        #X = R
        
        X = self.drop(X)
        X, hidden = self.rnn(X, hidden)
        X = self.drop(X)
        
        X = X.view(X.size(0)*X.size(1), X.size(2))
        
        X = self.decoder(X)
        log_probs = torch.log_softmax(X, -1)
        return log_probs, hidden
      

In [12]:
def repackage_hidden(h):
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [14]:
def evaluate(data_source, criterion):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss_e = 0
    total_number_of_words = 0

    with torch.no_grad():
        for i, batch in enumerate(data_source):
            labels = batch.theme.cuda().long() - 1
            batch = batch.review_text.cuda().long()
            hidden = None
            if batch.shape[0] > 2:
                data, targets = batch[:-1,:], batch[1:,:]
                number_of_words = data.shape[0] * data.shape[1]
                
                output, hidden = model(data, labels, hidden)
                output_flat = output.contiguous().view(-1, vocab_size)

                total_loss_e += criterion(output_flat, targets.contiguous().view(-1)).detach().item()
                total_number_of_words += number_of_words
                hidden = repackage_hidden(hidden)
            
    return total_loss_e / total_number_of_words

In [15]:
import numpy as np
import gc

In [16]:
def train(model, ep0, epN, train_iter, dev_iter, optimizer, criterion, 
          max_grad_norm, model_name, best_ppl = float('inf')):
    
    best_ppl = best_ppl
    
    len_train_iter = len(train_iter)
    for epoch in range(ep0, epN):
        model.train()
        total_loss_e = 0
        total_number_of_words = 0 
        
        for i, batch in enumerate(train_iter):

            labels = batch.theme.cuda().long() - 1
            batch = batch.review_text.cuda().long()
            hidden = None
            if batch.shape[0] > 2:
                data, targets = batch[:-1,:], batch[1:,:]
                
                number_of_words = data.shape[0] * data.shape[1]
                
                optimizer.zero_grad()
                output, hidden = model(data, labels, hidden)
                hidden = repackage_hidden(hidden)

                output_flat = output.contiguous().view(-1, vocab_size)
                epoch_loss = criterion(output_flat, targets.contiguous().view(-1))
                
                total_loss_e += epoch_loss.detach().item()
                total_number_of_words += number_of_words
            
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                epoch_loss.backward()
                optimizer.step()

                if i % 500 == 0:
                    cur_loss = epoch_loss.item() / number_of_words #np.mean(total_loss_e)#
                    tr_ppl_print = np.exp(cur_loss)
                    print("| epoch {:3d} | batch {} / {} | train_loss {} | train_ppl {}".format(
                            epoch, i, len_train_iter, 
                            np.round(cur_loss, 3), np.round(tr_ppl_print, 3)))

                #gc.collect()
                #torch.cuda.empty_cache()
                
                if i == 10000: #len_train_iter - 1:
                    cur_loss = total_loss_e / total_number_of_words #np.mean(total_loss_e) #
                    tr_ppl_print = np.exp(cur_loss)
                    gc.collect()
                    torch.cuda.empty_cache()
                    val_loss_eval = evaluate(dev_iter, criterion)
                    val_ppl_print = np.exp(val_loss_eval)
                    
                    template = "| epoch {:3d} | batch {} / {} | train_loss {} | train_ppl {} | val_loss {} | val_ppl {}"
                    print(template.format(
                            epoch, i, len_train_iter, 
                            np.round(cur_loss, 3), np.round(tr_ppl_print, 3), 
                            np.round(val_loss_eval, 3), np.round(val_ppl_print, 3)))

                    if val_ppl_print < best_ppl :
                        print('old best ppl {} new best ppl {}'.format(best_ppl, val_ppl_print))
                        best_ppl = val_ppl_print
                        best_model_name = 'best_model_{}_{}.model'.format(model_name, best_ppl)
                        print('save model...', best_model_name)
                        with open(best_model_name, 'wb') as file:
                            torch.save(model, file) 

                    gc.collect()
                    model.train()
                    
                if i == 10000: break


In [17]:
#with open('./best_model_base_model_ppl_250.3359726270283.model', 'rb') as file:
#    model = torch.load(file)#BaseModel().cuda()
#    model.eval()

In [18]:
#with open('best_model_name', 'rb') as file:
#    model = torch.load(file) 

In [19]:
model = BaseModel(pretrained=True).cuda()

In [20]:
model

BaseModel(
  (drop): Dropout(p=0.5)
  (word_embedding): Embedding(20004, 300)
  (label_embedding): Embedding(5, 20)
  (rnn): LSTM(320, 1024, num_layers=2, dropout=0.5)
  (decoder): Linear(in_features=1024, out_features=20004, bias=True)
)

In [21]:
# implement PPL
learning_rate = 0.001
criterion = nn.NLLLoss(reduction='sum',
                       ignore_index=train_dataset.fields["review_text"].vocab.stoi['<pad>']).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train(model,
      ep0 = 0,
      epN = 5,
      train_iter = train_iter,
      dev_iter = val_iter,
      optimizer = optimizer,
      criterion = criterion,
      max_grad_norm = 10,
      model_name = 'base_model_ppl',
      best_ppl = 26821.121)

| epoch   0 | batch 500 / 10981 | train_loss 3.953 | train_ppl 52.096
| epoch   0 | batch 1000 / 10981 | train_loss 3.984 | train_ppl 53.754
| epoch   0 | batch 1500 / 10981 | train_loss 4.302 | train_ppl 73.875
| epoch   0 | batch 2000 / 10981 | train_loss 4.159 | train_ppl 64.028
| epoch   0 | batch 2500 / 10981 | train_loss 3.813 | train_ppl 45.264
| epoch   0 | batch 3000 / 10981 | train_loss 4.225 | train_ppl 68.4
| epoch   0 | batch 3500 / 10981 | train_loss 4.37 | train_ppl 79.016
| epoch   0 | batch 4000 / 10981 | train_loss 4.125 | train_ppl 61.846
| epoch   0 | batch 4500 / 10981 | train_loss 4.484 | train_ppl 88.593
| epoch   0 | batch 5000 / 10981 | train_loss 4.374 | train_ppl 79.398
| epoch   0 | batch 5500 / 10981 | train_loss 4.665 | train_ppl 106.138
| epoch   0 | batch 6000 / 10981 | train_loss 4.377 | train_ppl 79.579
| epoch   0 | batch 6500 / 10981 | train_loss 4.616 | train_ppl 101.101
| epoch   0 | batch 7000 / 10981 | train_loss 4.514 | train_ppl 91.294
| epoch 

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BaseModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


| epoch   1 | batch 500 / 10981 | train_loss 3.26 | train_ppl 26.045
| epoch   1 | batch 1000 / 10981 | train_loss 3.451 | train_ppl 31.534
| epoch   1 | batch 1500 / 10981 | train_loss 3.696 | train_ppl 40.284
| epoch   1 | batch 2000 / 10981 | train_loss 3.563 | train_ppl 35.277
| epoch   1 | batch 2500 / 10981 | train_loss 3.353 | train_ppl 28.579
| epoch   1 | batch 3000 / 10981 | train_loss 3.767 | train_ppl 43.239
| epoch   1 | batch 3500 / 10981 | train_loss 3.896 | train_ppl 49.194
| epoch   1 | batch 4000 / 10981 | train_loss 3.699 | train_ppl 40.395


In [ ]:
train(model,
      ep0 = 1,
      epN = 10,
      train_iter = train_iter,
      dev_iter = val_iter,
      optimizer = optimizer,
      criterion = criterion,
      max_grad_norm = 10,
      model_name = 'base_model_ppl',
      best_ppl = 26821.121)

| epoch   0 | batch 500 / 1373 | train_loss 4.952 | train_ppl 141.401
| epoch   0 | batch 1000 / 1373 | train_loss 4.934 | train_ppl 138.885
| epoch   0 | batch 1372 / 1373 | train_loss 5.07 | train_ppl 159.23 | val_loss 4.853 | val_ppl 128.142
old best ppl 26821.121 new best ppl 128.142149838171
save model... best_model_base_model_ppl_128.142149838171.model


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BaseModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
